<a href="https://colab.research.google.com/github/ke-wen/ke-wen/blob/main/Milestone/Sample_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Reading the dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/AP_FYP")
!pwd


/content/drive/MyDrive/AP_FYP


In [3]:
import pandas as pd

In [4]:
# Step 1: Load the Dataset
dataset_path = 'data/HateSpeechDatasetBalanced.csv'
data = pd.read_csv(dataset_path)
print('The shape of the dataset is:', data.shape)

The shape of the dataset is: (726119, 2)


There is too much data, so we randomly select 1/10 of it.And apply the best performing model to whole data

In [5]:
df = data.sample(frac=0.1, random_state=42)
df

,Content,Label
615013,you should be deeply embarrassed... by not ful...,1
75913,do not make me make you fall in love with a bi...,0
523130,trump america is anti immigrant sexual activit...,1
682117,you guys are clearly a pole smoker please get ...,1
395535,oh come along the only reason people like stri...,0
...,...,...
386016,hi just a means cool a k a to say you that the...,0
439019,kiss i am not forgetting to update them its ju...,0
394958,mediation committee oppose per user page attac...,0
308990,pictures or it did not happen got to see so ma...,0


In [6]:
df.head(10)

,Content,Label
615013,you should be deeply embarrassed... by not ful...,1
75913,do not make me make you fall in love with a bi...,0
523130,trump america is anti immigrant sexual activit...,1
682117,you guys are clearly a pole smoker please get ...,1
395535,oh come along the only reason people like stri...,0
63523,did you just use it means cool a k a batty are...,0
299499,more versions according to there are some more...,0
270906,thank you for reverting my edit without postin...,0
181806,you are going to claim that heating up methane...,0
312861,tweeters have a fabulous weekend paparazzi,0


In [7]:
df["Label"].value_counts()

,count
Label,
1,36483
0,36129


2. Exploratory data analysis

In [8]:
print(df.isna().sum())

Content    0
Label      0
dtype: int64


In [9]:
!pip install transformers torch scikit-learn

4. Train - Validation Split

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(df["Content"], df['Label'], test_size=0.1, random_state=42)

Use BERT for text embeddings:

In [11]:
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm


# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BERT-based Classifier
# Load BERT tokenizer and model, move to GPU
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)

# Tokenize and encode the text data, move to GPU
max_length = 128
X_train_tokens = tokenizer(list(train_data), truncation=True, padding=True, max_length=max_length, return_tensors="pt", add_special_tokens=True).to(device)
X_test_tokens = tokenizer(list(test_data), truncation=True, padding=True, max_length=max_length, return_tensors="pt", add_special_tokens=True).to(device)

# Calculate BERT embeddings for the text data
def get_bert_embeddings(tokens):
    embeddings = []
    for i in tqdm(range(len(tokens['input_ids']))):
        with torch.no_grad():
            output = bert_model(input_ids=tokens['input_ids'][i].unsqueeze(0), attention_mask=tokens['attention_mask'][i].unsqueeze(0))
        embeddings.append(output[0].squeeze().mean(dim=0).cpu().numpy())
    return embeddings


X_train_bert_embeddings = get_bert_embeddings(X_train_tokens)
X_test_bert_embeddings = get_bert_embeddings(X_test_tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 7262/7262 [01:14<00:00, 97.41it/s]


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_bert_embeddings, train_labels)

NB = GaussianNB()
NB.fit(X_train_bert_embeddings, train_labels)

# Train a classifier on BERT embeddings (you can use any classifier of your choice)
# Here, we'll use Logistic Regression as an example
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_bert_embeddings, train_labels)

#rf=RandomForestClassifier()
#rf.fit(train_embeddings, train_labels)
#xgb=GradientBoostingClassifier()
#xgb.fit(train_embeddings, train_labels)

LogisticRegression(max_iter=1000)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the test data
predictions = knn.predict(X_test_bert_embeddings)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')
#predictions = rf.predict(test_embeddings)
predictions = NB.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Naive Bais Accuracy: {accuracy * 100:.2f}%')

#predictions = rf.predict(test_embeddings)
predictions = lr.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Logistic Regression Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 79.19%
Naive Bais Accuracy: 70.99%
Logistic Regression Accuracy: 80.65%


In [14]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the test data
predictions = knn.predict(X_test_bert_embeddings)
# Create a classification report
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])

# Create a confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

# Print the classification report and confusion matrix
print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)

#predictions = rf.predict(test_embeddings)
predictions = NB.predict(X_test_bert_embeddings)
# Create a classification report
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])

# Create a confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

# Print the classification report and confusion matrix
print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)

#predictions = rf.predict(test_embeddings)
predictions = lr.predict(X_test_bert_embeddings)
# Create a classification report
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])

# Create a confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

# Print the classification report and confusion matrix
print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)




Classification Report:
              precision    recall  f1-score   support

    negative       0.81      0.76      0.79      3621
    positive       0.78      0.82      0.80      3641

    accuracy                           0.79      7262
   macro avg       0.79      0.79      0.79      7262
weighted avg       0.79      0.79      0.79      7262


Confusion Matrix:
[[2759  862]
 [ 649 2992]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.77      0.60      0.67      3621
    positive       0.67      0.82      0.74      3641

    accuracy                           0.71      7262
   macro avg       0.72      0.71      0.71      7262
weighted avg       0.72      0.71      0.71      7262


Confusion Matrix:
[[2165 1456]
 [ 651 2990]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.82      0.79      0.80      3621
    positive       0.80      0.82      0.81      3641

    accuracy         

Tuning:Try More iterations for Logistic Regression

In [15]:
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train_bert_embeddings, train_labels)
#predictions = rf.predict(test_embeddings)
predictions = lr.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Logistic Regression Accuracy: {accuracy * 100:.2f}%')

Logistic Regression Accuracy: 80.65%


No significant improvement

try diferent k

In [16]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_bert_embeddings, train_labels)
predictions = knn.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 77.94%


In [17]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_bert_embeddings, train_labels)
predictions = knn.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 79.34%


In [22]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train_bert_embeddings, train_labels)
predictions = knn.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 79.48%


No significant improvement

Change feature extractor to TF

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Step 1: Use Count Vectorizer (TF) to transform the text data
vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tf = vectorizer.fit_transform(train_data)
X_test_tf = vectorizer.transform(test_data)

# Step 2: Train and evaluate KNN model
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_tf, train_labels)
knn_predictions = knn.predict(X_test_tf)
knn_accuracy = accuracy_score(test_labels, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy * 100:.2f}%')
print("KNN Classification Report:")
print(classification_report(test_labels, knn_predictions))

# Step 3: Train and evaluate Naive Bayes model
nb = MultinomialNB()
nb.fit(X_train_tf, train_labels)
nb_predictions = nb.predict(X_test_tf)
nb_accuracy = accuracy_score(test_labels, nb_predictions)
print(f'Naive Bayes Accuracy: {nb_accuracy * 100:.2f}%')
print("Naive Bayes Classification Report:")
print(classification_report(test_labels, nb_predictions))

# Step 4: Train and evaluate Logistic Regression model
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train_tf, train_labels)
lr_predictions = lr.predict(X_test_tf)
lr_accuracy = accuracy_score(test_labels, lr_predictions)
print(f'Logistic Regression Accuracy: {lr_accuracy * 100:.2f}%')
print("Logistic Regression Classification Report:")
print(classification_report(test_labels, lr_predictions))


KNN Accuracy: 66.74%
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.72      0.68      3621
           1       0.69      0.61      0.65      3641

    accuracy                           0.67      7262
   macro avg       0.67      0.67      0.67      7262
weighted avg       0.67      0.67      0.67      7262

Naive Bayes Accuracy: 76.38%
Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.62      0.72      3621
           1       0.71      0.90      0.79      3641

    accuracy                           0.76      7262
   macro avg       0.79      0.76      0.76      7262
weighted avg       0.79      0.76      0.76      7262

Logistic Regression Accuracy: 80.07%
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      3621
           1       0.79      0.82      0.80  

Change feature extractor to TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Step 1: Use TF-IDF to transform the text data
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(train_data)
X_test_tfidf = vectorizer.transform(test_data)

# Step 2: Train and evaluate KNN model
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_tfidf, train_labels)
knn_predictions = knn.predict(X_test_tfidf)
knn_accuracy = accuracy_score(test_labels, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy * 100:.2f}%')
print("KNN Classification Report:")
print(classification_report(test_labels, knn_predictions))

# Step 3: Train and evaluate Naive Bayes model
nb = MultinomialNB()
nb.fit(X_train_tfidf, train_labels)
nb_predictions = nb.predict(X_test_tfidf)
nb_accuracy = accuracy_score(test_labels, nb_predictions)
print(f'Naive Bayes Accuracy: {nb_accuracy * 100:.2f}%')
print("Naive Bayes Classification Report:")
print(classification_report(test_labels, nb_predictions))

# Step 4: Train and evaluate Logistic Regression model
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train_tfidf, train_labels)
lr_predictions = lr.predict(X_test_tfidf)
lr_accuracy = accuracy_score(test_labels, lr_predictions)
print(f'Logistic Regression Accuracy: {lr_accuracy * 100:.2f}%')
print("Logistic Regression Classification Report:")
print(classification_report(test_labels, lr_predictions))


KNN Accuracy: 56.87%
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.55      0.56      3621
           1       0.57      0.59      0.58      3641

    accuracy                           0.57      7262
   macro avg       0.57      0.57      0.57      7262
weighted avg       0.57      0.57      0.57      7262

Naive Bayes Accuracy: 77.28%
Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.69      0.75      3621
           1       0.73      0.86      0.79      3641

    accuracy                           0.77      7262
   macro avg       0.78      0.77      0.77      7262
weighted avg       0.78      0.77      0.77      7262

Logistic Regression Accuracy: 80.18%
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      3621
           1       0.80      0.81      0.80  

BERT embeddings are the best feature extractor. Next, I will apply it to the entire dataset.